<a href="https://colab.research.google.com/github/mak109/cs6910_assignment2/blob/main/PART%20B/cs6910_assignment2_partB_question1_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Fine-tuning pretrained Models

## 1. Packages and imports

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import datetime
import shutil
from zipfile import ZipFile
from PIL import Image
from inspect import *
from matplotlib import gridspec
plt.rcParams["figure.figsize"] = (20,10)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import * 
autotune = tf.data.AUTOTUNE
from functools import reduce
import random
import uuid
random.seed(123)

## 2. UTA-RLDD preprocessed dataset downloaded from kaggle

In [12]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
#give kaggle username and key to download dataset
api_token = {"username":"","key":""}


import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mak1999/uta-rldd-2


100%|█████████████████████████████████████▉| 2.40G/2.40G [02:04<00:00, 20.2MB/s]
100%|██████████████████████████████████████| 2.40G/2.40G [02:04<00:00, 20.7MB/s]


In [13]:
foldername = 'uta-rldd-2.zip'
with ZipFile(foldername, 'r') as z:
    print('Extracting all the folders now...')
    z.extractall()
    print('Done!')
os.remove(foldername)

Extracting all the folders now...
Done!


## 3. Wandb setup for hyperparameter tuning

In [5]:
import wandb
os.environ['WANDB_ENTITY'] = 'ipda526'
os.environ['WANDB_PROJECT'] = 'finetune-drowsiness-detection'
#wandb key used for storing model in wandb
wandb.login(key='')
from wandb.keras import WandbCallback,WandbMetricsLogger, WandbModelCheckpoint

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 4. Model definition and training

In [6]:
image_size = (256,256)
num_classes = 3 #0 - awake 1-drowsy 2 - low vigilant
train_dir = 'UTA-RLDD/train'
val_dir = 'UTA-RLDD/val'

In [7]:
#Creating dictionary of models based on imagenet 
model_list = dict()
for key,value in getmembers(tf.keras.applications,isfunction):
    model_list[key] = value
    
print(model_list)

{'ConvNeXtBase': <function ConvNeXtBase at 0x7351b6d228c0>, 'ConvNeXtLarge': <function ConvNeXtLarge at 0x7351b6d22950>, 'ConvNeXtSmall': <function ConvNeXtSmall at 0x7351b6d22830>, 'ConvNeXtTiny': <function ConvNeXtTiny at 0x7351b6d227a0>, 'ConvNeXtXLarge': <function ConvNeXtXLarge at 0x7351b6d229e0>, 'DenseNet121': <function DenseNet121 at 0x7351b6d273b0>, 'DenseNet169': <function DenseNet169 at 0x7351b6d27440>, 'DenseNet201': <function DenseNet201 at 0x7351b6d274d0>, 'EfficientNetB0': <function EfficientNetB0 at 0x7351b6d2f0e0>, 'EfficientNetB1': <function EfficientNetB1 at 0x7351b6d2f170>, 'EfficientNetB2': <function EfficientNetB2 at 0x7351b6d2f200>, 'EfficientNetB3': <function EfficientNetB3 at 0x7351b6d2f290>, 'EfficientNetB4': <function EfficientNetB4 at 0x7351b6d2f320>, 'EfficientNetB5': <function EfficientNetB5 at 0x7351b6d2f3b0>, 'EfficientNetB6': <function EfficientNetB6 at 0x7351b6d2f440>, 'EfficientNetB7': <function EfficientNetB7 at 0x7351b6d2f4d0>, 'EfficientNetV2B0': <

In [8]:
#Creating model using pretrained model
def CNN(config,augmentation = None):
    base_model = model_list[config['model']](input_shape=image_size +(3,),include_top=False,weights='imagenet')
    base_model.trainable = True #this is important
    if(len(base_model.layers) > config['fine_tune_last']):
        for layer in base_model.layers[:-config['fine_tune_last']]:
            layer.trainable = False    
    global_average_layer = layers.GlobalAveragePooling2D()
    prediction_layer = layers.Dense(num_classes,activation='softmax')
    inputs = layers.Input((image_size[0],image_size[1],3))
    input_rescale=layers.Rescaling(1./255)(inputs)
    x = base_model(input_rescale)
    x = global_average_layer(x)
    x = layers.Dropout(config['dropout'])(x)
    outputs = prediction_layer(x)
    model = keras.Model(inputs,outputs)
    return model

In [9]:
# tf.debugging.set_log_device_placement(True)
tf.config.set_soft_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [10]:
def train(config_in = None,checkpointing=False):


  #Default parameters

    config_ = {
    "model": 'VGG19',
    "learning_rate": 1e-4,
    "data_augment": "True",
    "dropout":0.6,
    "batch_size":64,
    "fine_tune_last":10,
    "epochs":5
    }

    '''Wandb Configs'''
    wandb.init(config=config_)
    config = wandb.config
    #Setting run name for better readability
    wandb.run.name = "model_"+str(config["model"])+"bs_"+str(config["batch_size"])+"epochs_"+str(config["epochs"])+"fine_"+str(config['fine_tune_last'])
    #Removing the temporary train/val dir if existing
    shutil.rmtree(train_dir,ignore_errors=True)
    shutil.rmtree(val_dir,ignore_errors=True)
    x = random.randint(0,4)
    #Pick up a random fold and use it for validation and remaining other 4 folds for training
    for i in range(5):
        if i == x:
            print(f'Copying fold {x+1} to val...')
            shutil.copytree(f'UTA-RLDD/fold{x+1}','UTA-RLDD/val')
        else:
            print(f'Copying fold {i+1} to train...')
            fold = f'fold{i+1}'
            fold_path = os.path.join('UTA-RLDD', fold)
            for subdir, dirs, files in os.walk(fold_path):
                dest_subdir = subdir.replace(fold, 'train')
                if not os.path.exists(dest_subdir):
                    os.makedirs(dest_subdir)
                for file in files:
                    shutil.copy(os.path.join(subdir, file), os.path.join(dest_subdir, file))

    print('Done!')
    #Data Augmentation
    if config["data_augment"] == 'True':
        data_generator = ImageDataGenerator(
        rotation_range=50, #random rotation between -50(clockwise) to 50(anti-clockwise) degree
        brightness_range=(0.2,0.8), 
        zoom_range=0.3, #zoom in range from [0.7,1.3]
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1, #Horizontal Shifting as a ratio of width
        height_shift_range=0.2,#Vertical Shifting as a ratio of height
        data_format='channels_last'
#         
        )
    else:
        data_generator = ImageDataGenerator(
            data_format='channels_last'
        )
    #Train set creation after conditional augmentation
    train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = config['batch_size'],
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle=True,
    seed=123
    )
    val_generator = ImageDataGenerator(data_format='channels_last').flow_from_directory(
        val_dir,
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=True,
        seed=123
    
    )
    try:
        with tf.device('/device:GPU:0'):
            #Building Model based on config 
            model = CNN(config)
            #Early Stopping to prevent overfitting
            early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,start_from_epoch=3)
            #Compiling model 
            model.compile(
            optimizer=optimizers.Adam(learning_rate=config["learning_rate"]),
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
            )
            #For checkpointing default value is False
            if checkpointing == True:
                current_directory = os.getcwd()
                final_directory = os.path.join(current_directory, f'models_{datetime.datetime.now()}')
                if not os.path.exists(final_directory):
                    os.makedirs(final_directory)
                checkpoint_filepath = final_directory
                model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                  filepath=checkpoint_filepath,
                  save_weights_only=False,
                  monitor='val_accuracy',
                  mode='max',
                  save_best_only=True)
                  #Fitting Model
                history = model.fit(train_generator,
                  validation_data=val_generator,
                  epochs=config["epochs"],
                  verbose=1,
                  # callbacks = [WandbCallback()] #Used with wandb
                  callbacks = [early_stop_callback,model_checkpoint_callback] #Custom callback for checkpointing
                  )
            else:
                history = model.fit(train_generator,
                  validation_data=val_generator,
                  epochs=config["epochs"],
                  verbose=1,#WandbMetricsLogger(log_freq="epoch"),
                  callbacks = [early_stop_callback,WandbCallback(monitor='val_accuracy',mode='auto')] #Used with wandb
                  )
            
    except RuntimeError as e:
          print(e)
    wandb.finish()
    shutil.rmtree(train_dir,ignore_errors=True)
    shutil.rmtree(val_dir,ignore_errors=True)
    return history,model

## Standalone Training

In [14]:
history,model = train()
#Visualization part
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
# plt.savefig('metrics.jpg')
plt.show()

Copying fold 1 to train...
Copying fold 2 to train...
Copying fold 3 to val...
Copying fold 4 to train...
Copying fold 5 to train...
Done!
Found 7986 images belonging to 3 classes.
Found 2160 images belonging to 3 classes.
80134624/80134624 [==============================] - 4s 0us/step


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5


/opt/conda/lib/python3.7/site-packages/keras/backend.py:5586: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits, "Softmax", "sparse_categorical_crossentropy"


125/125 [==============================] - ETA: 0s - loss: 1.1070 - accuracy: 0.3343

wandb: Adding directory to artifact (/kaggle/working/wandb/run-20230429_051404-beo7mtw5/files/model-best)... Done. 1.1s


125/125 [==============================] - 362s 3s/step - loss: 1.1070 - accuracy: 0.3343 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 2/5
125/125 [==============================] - ETA: 0s - loss: 1.0987 - accuracy: 0.3385

wandb: Adding directory to artifact (/kaggle/working/wandb/run-20230429_051404-beo7mtw5/files/model-best)... Done. 1.2s


125/125 [==============================] - 339s 3s/step - loss: 1.0987 - accuracy: 0.3385 - val_loss: 1.0987 - val_accuracy: 0.3593
Epoch 3/5
125/125 [==============================] - 331s 3s/step - loss: 1.0986 - accuracy: 0.3361 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 4/5
125/125 [==============================] - 333s 3s/step - loss: 1.0985 - accuracy: 0.3423 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/5
125/125 [==============================] - 332s 3s/step - loss: 1.0986 - accuracy: 0.3393 - val_loss: 1.0986 - val_accuracy: 0.3333


accuracy,▁▅▃█▅
epoch,▁▃▅▆█
loss,█▁▁▁▁
val_accuracy,▁█▁▁▁
val_loss,█▅▅▁▁
accuracy,0.33934
best_epoch,1
best_val_accuracy,0.35926
epoch,4
loss,1.09858
val_accuracy,0.33333


## Wandb integration for hyperparameter tuning

In [ ]:
import uuid
'''Wandb Sweeps'''
sweep_config = {
  "name" : "best-sweep-finetune-kaggle"+str(uuid.uuid1()),
  "method" : "bayes",
  "metric" : {
      "name" : "val_accuracy",
      "goal" : "maximize"
  },
  
  "parameters" : {
      "model" : {
          "values" : ["InceptionV3", "InceptionResNetV2","Xception","ResNet50","MobileNetV2"]
      },

  "learning_rate" :{
      "values" : [1e-3,1e-4]
  },
  "data_augment" : {
      "values" : ["True","False"]
  },
  "dropout" : {
      "values" : [0.2,0.3,0.4]
  },

  "batch_size" : {
      "values" : [32,64]
  },
  "fine_tune_last" : {
  "values" : [0,10,20,30]
  },
    "epochs" : {
      "values" : [5,10,15,20]
    }
  }
}
sweep_id=wandb.sweep(sweep_config,entity="ipda526",project="finetune-drowsiness-detection")
wandb.agent(sweep_id, function=train, count=10) # For ten runs